# Load Required Libraries


In [3]:
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('conll2000')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\sagar\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


True

In [4]:
from xgboost import XGBClassifier

In [6]:
import spacy
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
import unicodedata
from contractions import CONTRACTION_MAP
nlp=spacy.load("en_core",parse=True,tag=True,entity=True)
tokenizer=ToktokTokenizer()
stopwords_list=set(nltk.corpus.stopwords.words("english"))
stopwords_list.remove("no")
stopwords_list.remove("not")
stopwords_list.remove("above")
stopwords_list.remove("up")
stopwords_list.remove("below")
stopwords_list.remove("down")
stopwords_list.remove("don't")
stopwords_list.remove("doesn't")
stopwords_list.remove("aren't")
stopwords_list.remove("didn't")
stopwords_list.remove("are")
stopwords_list.remove("now")
stopwords_list.remove("won")


# Read the Data 

In [22]:
data=pd.read_excel("D:\\DBDA Project\\Excels\\AllData.xlsx")
data.head(10)

,Date,company,news,loss/profit,%change
0,2019-01-14,HCL,"HCL Tech Q3 PAT seen up 4.2% QoQ to Rs. 2,629....",0,-0.472399
1,2018-12-07,HCL,HCL Technologies-IBM deal fails to enthuse inv...,0,-3.941059
2,2018-12-06,HCL,Technical Views | Top buy & sell ideas by Ashw...,0,-3.614630
3,2018-11-09,HCL,HCL Technologies Limited Q2 FY’19 Earnings Con...,0,-0.097087
4,2018-10-29,HCL,"Buy HCL Technologies, target Rs 1182: Anand Rathi",1,2.113990
5,2018-10-24,HCL,HCL Technologies – good outlook backed by reas...,1,0.775154
6,2018-10-23,HCL,HCL Technologies sees 5.7% sequential rise in ...,0,-1.778351
7,2018-10-22,HCL,HCL Technologies Q2 results on October 23; her...,1,2.088542
8,2018-10-15,HCL,Stock Picks of the Day | Nifty may be vulnerab...,1,1.833232
9,2018-10-08,HCL,HCL Tech to set up global IT centres in Andhra...,0,-0.097674


# The Next Step is to Clean the Data
## We Remove Special Character
## We Use Lemmitization in our Program that will convert each word to its lemma
## We then Remove Stopwords

In [11]:
def remove_special_character(text,remove_digits=False):
    pattern=r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    TCS=re.sub(pattern,'',text)
    return text



In [12]:
def stemmer(text):
        ps=nltk.porter.PorterStemmer()
        TCS=' '.join([ps.stem(word) for word in TCS.split()])
        return text

In [13]:
def lemme(text):
    text=nlp(text)
    text=" ".join([word.lemma_ if word.lemma_ != "-PRON" else word.text for word in text ])
    return text
lemme("How US doing Bro?")

'how us do bro ?'

In [14]:
def remove_stopwords(text,is_lower_case=False):
    tokens=tokenizer.tokenize(text)
    tokens=[token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens=[token for token in tokens if token not in stopwords_list ]
    else:
        filtered_tokens=[token for token in tokens if token.lower() not in stopwords_list]
    filtered_text=" ".join(filtered_tokens)
    return filtered_text

remove_stopwords("you and me up now below ,without this world")
    

'up now below , without world'

In [15]:

def normalize_corpus(corpus,text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemme(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_character(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [9]:
data["Processed_news"]=normalize_corpus(data["news"]) #Adding new Column and calling the function 
norm_corpus=list(data["Processed_news"])
#showing sample
data.iloc[6][["Processed_news","news"]].to_dict()

{'Processed_news': 'hcl technology see 5 . 7 % sequential rise q2 net profit rs 2,540 crore',
 'news': 'HCL Technologies sees 5.7% sequential rise in Q2 net profit to Rs 2,540 crore'}

In [10]:
data.to_excel("D:\\DBDA Project\\Excels\\AllDataProcessed2.xlsx",index=False,encoding='utf-8')

#  Tagging Part of Speech

In [16]:
corpus = normalize_corpus(data['news'], text_lower_case=False, 
                          text_lemmatization=False, special_char_removal=False)

sentence = str(data.iloc[1].news)
sentence_nlp = nlp(sentence)
 
spacy_pos_tagged = [(word, word.tag_, word.pos_) for word in sentence_nlp]
pd.DataFrame(spacy_pos_tagged, columns=['Word', 'POS tag', 'Tag type'])


nltk_pos_tagged = nltk.pos_tag(sentence.split())
pd.DataFrame(nltk_pos_tagged, columns=['Word', 'POS tag'])

,Word,POS tag
0,HCL,NNP
1,Technologies-IBM,NNP
2,deal,NN
3,fails,VBZ
4,to,TO
5,enthuse,VB
6,investors.,JJ
7,HCL,NNP
8,Tech’s,NNP
9,$1.8-bn,JJ


# Sahllow parsing 

In [17]:
from nltk.corpus import conll2000

data_con = conll2000.chunked_sents()
train_data_con = data_con[:10900]
test_data_con = data_con[10900:] 

print(len(train_data_con), len(test_data_con))
print(train_data_con[1]) 

10900 48
(S
  Chancellor/NNP
  (PP of/IN)
  (NP the/DT Exchequer/NNP)
  (NP Nigel/NNP Lawson/NNP)
  (NP 's/POS restated/VBN commitment/NN)
  (PP to/TO)
  (NP a/DT firm/NN monetary/JJ policy/NN)
  (VP has/VBZ helped/VBN to/TO prevent/VB)
  (NP a/DT freefall/NN)
  (PP in/IN)
  (NP sterling/NN)
  (PP over/IN)
  (NP the/DT past/JJ week/NN)
  ./.)


# Dependency Parsing 

In [18]:
from spacy import displacy

displacy.render(sentence_nlp, jupyter=True, 
                options={'distance': 110,
                         'arrow_stroke': 2,
                         'arrow_width': 8})

#  Named Entity Recognition

In [19]:

sentence = str(data.iloc[1].news)
sentence_nlp = nlp(sentence)

# print named entities in article
print([(word, word.ent_type_) for word in sentence_nlp if word.ent_type_])

# visualize named entities
displacy.render(sentence_nlp, style='ent', jupyter=True)


[(HCL, 'ORG'), (Technologies, 'ORG'), (-, 'ORG'), (IBM, 'ORG'), (HCL, 'ORG'), (Tech, 'ORG'), (’s, 'ORG'), (1.8-bn, 'MONEY'), (IBM, 'ORG'), (HCL, 'ORG'), (Technologies, 'ORG'), (IBM, 'ORG'), ($, 'MONEY'), (1.8, 'MONEY'), (bln, 'MONEY')]


In [20]:

named_entities = []
for sentence in corpus:
    temp_entity_name = ''
    temp_named_entity = None
    sentence = nlp(sentence)
    for word in sentence:
        term = word.text 
        tag = word.ent_type_
        if tag:
            temp_entity_name = ' '.join([temp_entity_name, term]).strip()
            temp_named_entity = (temp_entity_name, tag)
        else:
            if temp_named_entity:
                named_entities.append(temp_named_entity)
                temp_entity_name = ''
                temp_named_entity = None

entity_frame = pd.DataFrame(named_entities, 
                            columns=['Entity Name', 'Entity Type'])


In [21]:
top_entities = (entity_frame.groupby(by=['Entity Name', 'Entity Type'])
                           .size()
                           .sort_values(ascending=False)
                           .reset_index().rename(columns={0 : 'Frequency'}))
top_entities.T.iloc[:,:15]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Entity Name,TCS,Wipro,Infosys,RIL,HDFC,Bank,Reliance Industries,HCL Tech,Kotak Mahindra Bank,HCL Technologies,Reliance,YES Bank,HCL Tech,India,Sukhani
Entity Type,ORG,ORG,ORG,ORG,ORG,ORG,ORG,ORG,ORG,ORG,ORG,ORG,PRODUCT,GPE,GPE
Frequency,1888,1342,1246,765,759,747,741,684,458,446,336,287,193,178,166


In [23]:
top_entities = (entity_frame.groupby(by=['Entity Type'])
                           .size()
                           .sort_values(ascending=False)
                           .reset_index().rename(columns={0 : 'Frequency'}))
top_entities.T.iloc[:,:15]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Entity Type,ORG,CARDINAL,PERSON,GPE,PERCENT,DATE,PRODUCT,NORP,MONEY,LOC,ORDINAL,FAC,EVENT,WORK_OF_ART,QUANTITY
Frequency,18849,5066,4849,1613,1379,878,820,470,356,145,95,80,52,45,43
